# Code seperated by cells

In [1]:
import numpy as np
import pandas as pd
import optuna
from functools import partial

In [2]:
# =====================================================
# Utility Functions
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:  # test data
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:  # train data
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data_with_poi():
    """Load competition data including POI features."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/new_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    poi = pd.read_csv(f'{path}/sector_poi.csv')
    return train, test, poi

In [3]:
# =====================================================
# Data Transformation
# =====================================================

def build_amount_matrix(train: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Pivot training data into [time x sector_id] transaction matrix."""
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_new_house_transactions', fill_value=0
    )

    # Ensure all 96 sectors are present
    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)

    return pivot

In [4]:
# =====================================================
# Modeling Helpers
# =====================================================

def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply December adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


def predict_one_step(a_hist, n_lags, alpha):
    """Predict next-step values for all sectors."""
    return pd.Series({
        sector: ewgm_per_sector(a_hist, sector, n_lags, alpha)
        if a_hist[sector].tail(n_lags).min() > 0 else 0.0
        for sector in a_hist.columns
    })


def evaluate_params(a_tr_full, n_lags, alpha, t2, clip_low, clip_high, val_len=6):
    """Evaluate parameters via rolling-origin backtest."""
    times = a_tr_full.index
    if len(times) < max(n_lags + 1, t2 + 1) + val_len:
        return 1e12

    rmses = []
    for t in times[-val_len:]:
        a_hist = a_tr_full.loc[a_tr_full.index < t]
        if len(a_hist) < max(n_lags, t2):
            continue

        y_true = a_tr_full.loc[t]
        y_pred = predict_one_step(a_hist, n_lags, alpha)

        if t % 12 == 11:  # December bump
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)

        rmses.append(np.sqrt(np.mean((y_pred - y_true) ** 2)))

    return float(np.mean(rmses)) if rmses else 1e12


def predict_horizon(a_tr, n_lags, alpha, t2):
    """Forecast horizon [67..78]."""
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)

    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            preds[sector] = ewgm_per_sector(a_tr, sector, n_lags, alpha)

    preds.index.name = 'time'
    return preds

In [5]:
# =====================================================
# Submission
# =====================================================

def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)

    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


def generate_submission_with_december_bump(n_lags=6, alpha=0.5, t2=6, clip_low=0.85, clip_high=1.4):
    """End-to-end pipeline for submission with December bump."""
    month_codes = build_month_codes()
    train, test = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    a_pred = predict_horizon(a_tr, n_lags, alpha, t2)

    # Apply December bump
    mult = compute_december_multipliers(a_tr, clip_low=clip_low, clip_high=clip_high)
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], mult)

    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('/Users/nikola/Python/KaggleCompetition/output/submission.csv', index=False)
    return a_tr, a_pred, sub

In [6]:
# =====================================================
# Optuna Optimization
# =====================================================

def optuna_objective(trial, a_tr):
    """Objective for Optuna hyperparameter tuning."""
    n_lags = trial.suggest_int('n_lags', 3, 12)
    alpha = trial.suggest_float('alpha', 0.20, 0.95)
    t2 = trial.suggest_int('t2', 3, 9)
    clip_low = trial.suggest_float('clip_low', 0.70, 0.95)
    clip_high = trial.suggest_float('clip_high', 1.10, 1.80)

    if clip_low >= clip_high:
        clip_low = max(0.70, clip_high - 0.05)

    return evaluate_params(a_tr, n_lags, alpha, t2, clip_low, clip_high)


def run_optuna_search(a_tr, n_trials=1000, seed=1337):
    """Run Optuna search and return the study."""
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(partial(optuna_objective, a_tr=a_tr), n_trials=n_trials, show_progress_bar=False)
    return study

In [7]:
def main():
    # Option 1: POI-enhanced submission (try this first)
    generate_submission_poi_enhanced(
        n_lags=6, 
        alpha=0.5, 
        t2=6, 
        clip_low=0.85, 
        clip_high=1.4,
        poi_weight=0.1
    )
    print("POI-enhanced submission saved")
    
    # Option 2: If you want to optimize hyperparameters with POI
    # month_codes = build_month_codes()
    # train, _, poi = load_competition_data_with_poi()
    # a_tr, poi_features = build_amount_matrix_with_poi(train, poi, month_codes)
    # study = run_optuna_search(a_tr, n_trials=1000, seed=1337)
    # print("Best parameters:", study.best_params)
    # generate_submission_poi_enhanced(**study.best_params, poi_weight=0.1)


if __name__ == "__main__":
    main()

NameError: name 'generate_submission_poi_enhanced' is not defined

# EWGM Complete code in one cell - Score: 0.56006

In [ ]:
import numpy as np
import pandas as pd
import optuna
from functools import partial

# =====================================================
# Utility Functions
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:  # test data
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:  # train data
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data():
    """Load competition training and test datasets."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/new_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    return train, test


# =====================================================
# Data Transformation
# =====================================================

def build_amount_matrix(train: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Pivot training data into [time x sector_id] transaction matrix."""
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_new_house_transactions', fill_value=0
    )

    # Ensure all 96 sectors are present
    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)

    return pivot


# =====================================================
# Modeling Helpers
# =====================================================

def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply December adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


def predict_one_step(a_hist, n_lags, alpha):
    """Predict next-step values for all sectors."""
    return pd.Series({
        sector: ewgm_per_sector(a_hist, sector, n_lags, alpha)
        if a_hist[sector].tail(n_lags).min() > 0 else 0.0
        for sector in a_hist.columns
    })


def evaluate_params(a_tr_full, n_lags, alpha, t2, clip_low, clip_high, val_len=6):
    """Evaluate parameters via rolling-origin backtest."""
    times = a_tr_full.index
    if len(times) < max(n_lags + 1, t2 + 1) + val_len:
        return 1e12

    rmses = []
    for t in times[-val_len:]:
        a_hist = a_tr_full.loc[a_tr_full.index < t]
        if len(a_hist) < max(n_lags, t2):
            continue

        y_true = a_tr_full.loc[t]
        y_pred = predict_one_step(a_hist, n_lags, alpha)

        if t % 12 == 11:  # December bump
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)

        rmses.append(np.sqrt(np.mean((y_pred - y_true) ** 2)))

    return float(np.mean(rmses)) if rmses else 1e12


def predict_horizon(a_tr, n_lags, alpha, t2):
    """Forecast horizon [67..78]."""
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)

    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            preds[sector] = ewgm_per_sector(a_tr, sector, n_lags, alpha)

    preds.index.name = 'time'
    return preds

def validate_ewgm_mape(a_tr, n_lags=6, alpha=0.5, t2=6, clip_low=0.85, clip_high=1.4, val_len=6):
    """Validate EWGM model on last val_len months and return average MAPE."""
    times = a_tr.index
    if len(times) < max(n_lags + 1, t2 + 1) + val_len:
        print("Not enough data for validation")
        return None
    
    mapes = []
    for t in times[-val_len:]:
        a_hist = a_tr.loc[a_tr.index < t]
        if len(a_hist) < max(n_lags, t2):
            continue
        
        y_true = a_tr.loc[t]
        y_pred = predict_one_step(a_hist, n_lags, alpha)
        
        # Apply December bump if needed
        if t % 12 == 11:
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)
        
        # Calculate MAPE (only for non-zero true values)
        mask = y_true > 0
        ape = np.abs((y_pred[mask] - y_true[mask]) / y_true[mask])
        mape = ape.mean() * 100
        mapes.append(mape)
        
        print(f"Month {t}: MAPE = {mape:.2f}%")
    
    avg_mape = np.mean(mapes)
    print(f"\nAverage MAPE: {avg_mape:.2f}%")
    return avg_mape


# =====================================================
# Submission
# =====================================================

def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)

    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


def generate_submission_with_december_bump(n_lags=6, alpha=0.5, t2=6, clip_low=0.85, clip_high=1.4):
    """End-to-end pipeline for submission with December bump."""
    month_codes = build_month_codes()
    train, test = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    a_pred = predict_horizon(a_tr, n_lags, alpha, t2)

    # Apply December bump
    mult = compute_december_multipliers(a_tr, clip_low=clip_low, clip_high=clip_high)
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], mult)

    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('/Users/nikola/Python/KaggleCompetition/output/EWGM/EWGM_submission.csv', index=False)
    return a_tr, a_pred, sub


# =====================================================
# Optuna Optimization
# =====================================================

def optuna_objective(trial, a_tr):
    """Objective for Optuna hyperparameter tuning."""
    n_lags = trial.suggest_int('n_lags', 3, 12)
    alpha = trial.suggest_float('alpha', 0.20, 0.95)
    t2 = trial.suggest_int('t2', 3, 9)
    clip_low = trial.suggest_float('clip_low', 0.70, 0.95)
    clip_high = trial.suggest_float('clip_high', 1.10, 1.80)

    if clip_low >= clip_high:
        clip_low = max(0.70, clip_high - 0.05)

    return evaluate_params(a_tr, n_lags, alpha, t2, clip_low, clip_high)


def run_optuna_search(a_tr, n_trials=1000, seed=1337):
    """Run Optuna search and return the study."""
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(partial(optuna_objective, a_tr=a_tr), n_trials=n_trials, show_progress_bar=False)
    return study


# =====================================================
# Main
# =====================================================

def main():
    month_codes = build_month_codes()
    train, _ = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    
    # Step 1: Validate baseline parameters
    print("="*60)
    print("BASELINE VALIDATION (n_lags=6, alpha=0.5)")
    print("="*60)
    baseline_mape = validate_ewgm_mape(a_tr, n_lags=6, alpha=0.5, t2=6,
                                       clip_low=0.85, clip_high=1.4, val_len=6)
    
    # Step 2: Run Optuna optimization
    print("\n" + "="*60)
    print("RUNNING OPTUNA HYPERPARAMETER OPTIMIZATION (512 trials)")
    print("="*60)
    study = run_optuna_search(a_tr, n_trials=512, seed=1337)
    best = study.best_params
    print(f"\nBest parameters: {best}")
    print(f"Best RMSE: {study.best_value:.2f}")
    
    # Step 3: Validate optimized parameters
    print("\n" + "="*60)
    print("OPTIMIZED PARAMETERS VALIDATION")
    print("="*60)
    optimized_mape = validate_ewgm_mape(a_tr, **best, val_len=6)
    
    # Step 4: Compare and generate submission
    print("\n" + "="*60)
    print("COMPARISON")
    print("="*60)
    print(f"Baseline MAPE: {baseline_mape:.2f}%")
    print(f"Optimized MAPE: {optimized_mape:.2f}%")
    print(f"Improvement: {baseline_mape - optimized_mape:.2f}%")
    
    # Step 5: Generate submission with best parameters
    print("\n" + "="*60)
    print("GENERATING SUBMISSION WITH OPTIMIZED PARAMETERS")
    print("="*60)
    generate_submission_with_december_bump(**best)
    print("Submission saved to /Users/nikola/Python/KaggleCompetition/output/submission.csv")


if __name__ == "__main__":
    main()

[I 2025-09-30 11:44:07,199] A new study created in memory with name: no-name-463cdda1-62b9-44cd-bbd9-8b6cd4b1097a
[I 2025-09-30 11:44:07,215] Trial 0 finished with value: 21383.972104866807 and parameters: {'n_lags': 5, 'alpha': 0.31901297911584925, 't2': 4, 'clip_low': 0.8148292218036416, 'clip_high': 1.3247003783641171}. Best is trial 0 with value: 21383.972104866807.
[I 2025-09-30 11:44:07,232] Trial 1 finished with value: 21231.50264119316 and parameters: {'n_lags': 8, 'alpha': 0.3964571941738585, 't2': 9, 'clip_low': 0.8832036381726205, 'clip_high': 1.1806919586782048}. Best is trial 1 with value: 21231.50264119316.
[I 2025-09-30 11:44:07,249] Trial 2 finished with value: 21112.848609130506 and parameters: {'n_lags': 6, 'alpha': 0.6713758846547837, 't2': 3, 'clip_low': 0.9458871512859103, 'clip_high': 1.4102574080515897}. Best is trial 2 with value: 21112.848609130506.
[I 2025-09-30 11:44:07,264] Trial 3 finished with value: 21122.422935786373 and parameters: {'n_lags': 10, 'alpha

BASELINE VALIDATION (n_lags=6, alpha=0.5)
Month 61: MAPE = 168.90%
Month 62: MAPE = 49.99%
Month 63: MAPE = 62.78%
Month 64: MAPE = 49.14%
Month 65: MAPE = 46.38%
Month 66: MAPE = 75.13%

Average MAPE: 75.39%

RUNNING OPTUNA HYPERPARAMETER OPTIMIZATION (512 trials)


[I 2025-09-30 11:44:07,397] Trial 11 finished with value: 21094.005759207124 and parameters: {'n_lags': 7, 'alpha': 0.6136437470111018, 't2': 7, 'clip_low': 0.9480450968302885, 'clip_high': 1.5953497173720117}. Best is trial 10 with value: 21093.690894088948.
[I 2025-09-30 11:44:07,415] Trial 12 finished with value: 21096.721246042234 and parameters: {'n_lags': 7, 'alpha': 0.5926902717110654, 't2': 7, 'clip_low': 0.9442375014368745, 'clip_high': 1.6138738076887598}. Best is trial 10 with value: 21093.690894088948.
[I 2025-09-30 11:44:07,434] Trial 13 finished with value: 21700.647970117716 and parameters: {'n_lags': 8, 'alpha': 0.20563828697295955, 't2': 7, 'clip_low': 0.9050039798962632, 'clip_high': 1.6018220250780288}. Best is trial 10 with value: 21093.690894088948.
[I 2025-09-30 11:44:07,454] Trial 14 finished with value: 21080.031173804742 and parameters: {'n_lags': 8, 'alpha': 0.5998900635155954, 't2': 8, 'clip_low': 0.9115106298560189, 'clip_high': 1.7977978030395279}. Best is 


Best parameters: {'n_lags': 9, 'alpha': 0.6741577959564671, 't2': 4, 'clip_low': 0.8798137129337238, 'clip_high': 1.205522354649888}
Best RMSE: 21046.09

OPTIMIZED PARAMETERS VALIDATION
Month 61: MAPE = 184.56%
Month 62: MAPE = 49.47%
Month 63: MAPE = 62.91%
Month 64: MAPE = 53.41%
Month 65: MAPE = 48.01%
Month 66: MAPE = 70.64%

Average MAPE: 78.17%

COMPARISON
Baseline MAPE: 75.39%
Optimized MAPE: 78.17%
Improvement: -2.78%

GENERATING SUBMISSION WITH OPTIMIZED PARAMETERS
Submission saved to /Users/nikola/Python/KaggleCompetition/output/submission.csv


# Code with POI - Score: 0.56569

In [18]:
import pandas as pd
import numpy as np
from functools import partial

# =====================================================
# Utility Functions
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:  # test data
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:  # train data
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data_with_poi():
    """Load competition data including POI features."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/new_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    poi = pd.read_csv(f'{path}/train/sector_poi.csv')
    return train, test, poi


# =====================================================
# Data Transformation
# =====================================================

def build_amount_matrix_with_poi(train: pd.DataFrame, poi: pd.DataFrame, month_codes: dict):
    """Build transaction matrix and POI features."""
    from sklearn.preprocessing import StandardScaler
    
    # Build transaction matrix
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_new_house_transactions', fill_value=0
    )
    
    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)
    
    # Prepare POI features - use actual column names from your CSV
    poi_cols = ['office_population', 'education', 'surrounding_housing_average_price', 
                'medical_health', 'subway_station_cnt']
    
    poi['sector_id'] = poi['sector'].str.extract(r'(\d+)').astype(int)
    poi_features = poi[['sector_id'] + poi_cols].set_index('sector_id')
    
    # Normalize POI features
    scaler = StandardScaler()
    poi_features[poi_cols] = scaler.fit_transform(poi_features[poi_cols])
    poi_features = poi_features.reindex(all_sectors, fill_value=0)
    
    return pivot, poi_features

# =====================================================
# Modeling Helpers
# =====================================================

def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply December adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


def ewgm_per_sector_with_poi(a_tr, poi_features, sector, n_lags, alpha, poi_weight=0.1):
    """Enhanced EWGM with POI adjustment."""
    base_pred = ewgm_per_sector(a_tr, sector, n_lags, alpha)
    if base_pred == 0:
        return 0.0
    
    poi_score = poi_features.loc[sector].sum()
    poi_multiplier = 1.0 + (poi_score * poi_weight)
    poi_multiplier = np.clip(poi_multiplier, 0.7, 1.3)
    
    return base_pred * poi_multiplier

def validate_model_with_poi(a_tr, poi_features, n_lags=6, alpha=0.5, t2=6, 
                            clip_low=0.85, clip_high=1.4, poi_weight=0.1, val_len=6):
    """
    Validate model on last val_len months using walk-forward validation.
    Returns average MAPE.
    """
    times = a_tr.index
    if len(times) < max(n_lags + 1, t2 + 1) + val_len:
        print("Not enough data for validation")
        return None
    
    mapes = []
    for t in times[-val_len:]:
        a_hist = a_tr.loc[a_tr.index < t]
        if len(a_hist) < max(n_lags, t2):
            continue
        
        y_true = a_tr.loc[t]
        
        # Predict using POI-enhanced model
        y_pred = pd.Series({
            sector: ewgm_per_sector_with_poi(a_hist, poi_features, sector, n_lags, alpha, poi_weight)
            if a_hist[sector].tail(n_lags).min() > 0 else 0.0
            for sector in a_hist.columns
        })
        
        # Apply December bump if needed
        if t % 12 == 11:
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)
        
        # Calculate MAPE
        mask = y_true > 0
        ape = np.abs((y_pred[mask] - y_true[mask]) / y_true[mask])
        mape = ape.mean() * 100
        mapes.append(mape)
        
        print(f"Month {t}: MAPE = {mape:.2f}%")
    
    avg_mape = np.mean(mapes)
    print(f"\nAverage MAPE: {avg_mape:.2f}%")
    return avg_mape


def predict_horizon_with_poi(a_tr, poi_features, n_lags, alpha, t2, poi_weight=0.1):
    """Forecast with POI-enhanced predictions."""
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)
    
    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            preds[sector] = ewgm_per_sector_with_poi(a_tr, poi_features, sector, n_lags, alpha, poi_weight)
    
    preds.index.name = 'time'
    return preds


# =====================================================
# Submission
# =====================================================

def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)
    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


def generate_submission_poi_enhanced(n_lags=6, alpha=0.5, t2=6, clip_low=0.85, clip_high=1.4, poi_weight=0.1):
    """End-to-end pipeline with POI features."""
    month_codes = build_month_codes()
    train, test, poi = load_competition_data_with_poi()
    
    a_tr, poi_features = build_amount_matrix_with_poi(train, poi, month_codes)
    a_pred = predict_horizon_with_poi(a_tr, poi_features, n_lags, alpha, t2, poi_weight)
    
    # Apply December bump
    mult = compute_december_multipliers(a_tr, clip_low=clip_low, clip_high=clip_high)
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], mult)
    
    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('/Users/nikola/Python/KaggleCompetition/output/EWGM_POI/EWGM_POI_submission_poi.csv', index=False)
    print(f"POI-enhanced submission saved (poi_weight={poi_weight})")
    return sub


# =====================================================
# Main
# =====================================================

def main():
    month_codes = build_month_codes()
    train, test, poi = load_competition_data_with_poi()
    a_tr, poi_features = build_amount_matrix_with_poi(train, poi, month_codes)
    
    print("="*60)
    print("VALIDATION - EWGM WITHOUT POI")
    print("="*60)
    mape = validate_model_with_poi(a_tr, poi_features, 
                                    n_lags=6, alpha=0.5, t2=6,
                                    clip_low=0.85, clip_high=1.4, 
                                    poi_weight=0.0,  # Disabled POI
                                    val_len=6)
    
    # Generate submission
    print("\n" + "="*60)
    print("GENERATING SUBMISSION")
    print("="*60)
    generate_submission_poi_enhanced(
        n_lags=6, alpha=0.5, t2=6, 
        clip_low=0.85, clip_high=1.4, 
        poi_weight=0.0  # Disabled POI
    )
    print("Submission saved")


if __name__ == "__main__":
    main()

VALIDATION - EWGM WITHOUT POI
Month 61: MAPE = 168.90%
Month 62: MAPE = 49.99%
Month 63: MAPE = 62.78%
Month 64: MAPE = 49.14%
Month 65: MAPE = 46.38%
Month 66: MAPE = 75.13%

Average MAPE: 75.39%

GENERATING SUBMISSION
POI-enhanced submission saved (poi_weight=0.0)
Submission saved


# Ensemble Code - Score: 0.56110

In [20]:
# =====================================================
# Utility Functions
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:  # test data
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:  # train data
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data():
    """Load competition training and test datasets."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/new_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    return train, test


# =====================================================
# Data Transformation
# =====================================================

def build_amount_matrix(train: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Pivot training data into [time x sector_id] transaction matrix."""
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_new_house_transactions', fill_value=0
    )

    # Ensure all 96 sectors are present
    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)

    return pivot


# =====================================================
# Modeling Helpers
# =====================================================

def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply December adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


def predict_one_step(a_hist, n_lags, alpha):
    """Predict next-step values for all sectors."""
    return pd.Series({
        sector: ewgm_per_sector(a_hist, sector, n_lags, alpha)
        if a_hist[sector].tail(n_lags).min() > 0 else 0.0
        for sector in a_hist.columns
    })


def evaluate_params(a_tr_full, n_lags, alpha, t2, clip_low, clip_high, val_len=6):
    """Evaluate parameters via rolling-origin backtest."""
    times = a_tr_full.index
    if len(times) < max(n_lags + 1, t2 + 1) + val_len:
        return 1e12

    rmses = []
    for t in times[-val_len:]:
        a_hist = a_tr_full.loc[a_tr_full.index < t]
        if len(a_hist) < max(n_lags, t2):
            continue

        y_true = a_tr_full.loc[t]
        y_pred = predict_one_step(a_hist, n_lags, alpha)

        if t % 12 == 11:  # December bump
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)

        rmses.append(np.sqrt(np.mean((y_pred - y_true) ** 2)))

    return float(np.mean(rmses)) if rmses else 1e12


def predict_horizon(a_tr, n_lags, alpha, t2):
    """Forecast horizon [67..78]."""
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)

    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            preds[sector] = ewgm_per_sector(a_tr, sector, n_lags, alpha)

    preds.index.name = 'time'
    return preds

def validate_ewgm_mape(a_tr, n_lags=6, alpha=0.5, t2=6, clip_low=0.85, clip_high=1.4, val_len=6):
    """Validate EWGM model on last val_len months and return average MAPE."""
    times = a_tr.index
    if len(times) < max(n_lags + 1, t2 + 1) + val_len:
        print("Not enough data for validation")
        return None
    
    mapes = []
    for t in times[-val_len:]:
        a_hist = a_tr.loc[a_tr.index < t]
        if len(a_hist) < max(n_lags, t2):
            continue
        
        y_true = a_tr.loc[t]
        y_pred = predict_one_step(a_hist, n_lags, alpha)
        
        # Apply December bump if needed
        if t % 12 == 11:
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)
        
        # Calculate MAPE (only for non-zero true values)
        mask = y_true > 0
        ape = np.abs((y_pred[mask] - y_true[mask]) / y_true[mask])
        mape = ape.mean() * 100
        mapes.append(mape)
        
        print(f"Month {t}: MAPE = {mape:.2f}%")
    
    avg_mape = np.mean(mapes)
    print(f"\nAverage MAPE: {avg_mape:.2f}%")
    return avg_mape


# =====================================================
# Submission
# =====================================================

def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)

    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


def generate_submission_with_december_bump(n_lags=6, alpha=0.5, t2=6, clip_low=0.85, clip_high=1.4):
    """End-to-end pipeline for submission with December bump."""
    month_codes = build_month_codes()
    train, test = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    a_pred = predict_horizon(a_tr, n_lags, alpha, t2)

    # Apply December bump
    mult = compute_december_multipliers(a_tr, clip_low=clip_low, clip_high=clip_high)
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], mult)

    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('/Users/nikola/Python/KaggleCompetition/output/EWGM/EWGM_submission.csv', index=False)
    return a_tr, a_pred, sub


def generate_ensemble_submission():
    """Ensemble of 3 EWGM models with different parameters."""
    month_codes = build_month_codes()
    train, test = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    
    print("Generating ensemble predictions...")
    
    # Model 1: Baseline parameters
    pred1 = predict_horizon(a_tr, n_lags=6, alpha=0.5, t2=6)
    print("Model 1 (baseline) complete")
    
    # Model 2: Optuna best parameters
    pred2 = predict_horizon(a_tr, n_lags=9, alpha=0.67, t2=4)
    print("Model 2 (optimized) complete")
    
    # Model 3: Conservative long-term parameters
    pred3 = predict_horizon(a_tr, n_lags=12, alpha=0.4, t2=8)
    print("Model 3 (conservative) complete")
    
    # Equal weight ensemble
    a_pred = (pred1 + pred2 + pred3) / 3
    print("Ensemble averaged")
    
    # Apply December bump
    mult = compute_december_multipliers(a_tr, clip_low=0.85, clip_high=1.4)
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], mult)
    
    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('/Users/nikola/Python/KaggleCompetition/output/ENSEMBLE_EWGM/submission_ensemble.csv', index=False)
    print("Ensemble submission saved")
    return sub


# =====================================================
# Optuna Optimization
# =====================================================

def optuna_objective(trial, a_tr):
    """Objective for Optuna hyperparameter tuning."""
    n_lags = trial.suggest_int('n_lags', 3, 12)
    alpha = trial.suggest_float('alpha', 0.20, 0.95)
    t2 = trial.suggest_int('t2', 3, 9)
    clip_low = trial.suggest_float('clip_low', 0.70, 0.95)
    clip_high = trial.suggest_float('clip_high', 1.10, 1.80)

    if clip_low >= clip_high:
        clip_low = max(0.70, clip_high - 0.05)

    return evaluate_params(a_tr, n_lags, alpha, t2, clip_low, clip_high)


def run_optuna_search(a_tr, n_trials=1000, seed=1337):
    """Run Optuna search and return the study."""
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(partial(optuna_objective, a_tr=a_tr), n_trials=n_trials, show_progress_bar=False)
    return study


# =====================================================
# Main
# =====================================================

def main():
    month_codes = build_month_codes()
    train, _ = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    
    # Validate baseline
    print("="*60)
    print("BASELINE VALIDATION (n_lags=6, alpha=0.5)")
    print("="*60)
    baseline_mape = validate_ewgm_mape(a_tr, n_lags=6, alpha=0.5, t2=6,
                                       clip_low=0.85, clip_high=1.4, val_len=6)
    
    # Generate ensemble submission
    print("\n" + "="*60)
    print("GENERATING ENSEMBLE SUBMISSION")
    print("="*60)
    generate_ensemble_submission()
    print("Ensemble submission saved to /Users/nikola/Python/KaggleCompetition/output/submission_ensemble.csv")


if __name__ == "__main__":
    main()

BASELINE VALIDATION (n_lags=6, alpha=0.5)
Month 61: MAPE = 168.90%
Month 62: MAPE = 49.99%
Month 63: MAPE = 62.78%
Month 64: MAPE = 49.14%
Month 65: MAPE = 46.38%
Month 66: MAPE = 75.13%

Average MAPE: 75.39%

GENERATING ENSEMBLE SUBMISSION
Generating ensemble predictions...
Model 1 (baseline) complete
Model 2 (optimized) complete
Model 3 (conservative) complete
Ensemble averaged
Ensemble submission saved
Ensemble submission saved to /Users/nikola/Python/KaggleCompetition/output/submission_ensemble.csv


# Ensemble with February and december bump - Score: 0.56110

In [28]:
import pandas as pd
import numpy as np
from functools import partial

# =====================================================
# Utility Functions
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:  # test data
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:  # train data
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data():
    """Load competition training and test datasets."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/new_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    return train, test


# =====================================================
# Data Transformation
# =====================================================

def build_amount_matrix(train: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Pivot training data into [time x sector_id] transaction matrix."""
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_new_house_transactions', fill_value=0
    )

    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)
    return pivot


# =====================================================
# Modeling Helpers
# =====================================================

def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def compute_february_multipliers(a_tr, eps=1e-9, min_feb_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level February multipliers (Chinese New Year effect)."""
    is_feb = (a_tr.index % 12 == 1)
    feb_means = a_tr[is_feb].mean()
    nonfeb_means = a_tr[~is_feb].mean()
    feb_counts = a_tr[is_feb].count()

    raw_mult = feb_means / (nonfeb_means + eps)
    overall_mult = float(feb_means.mean() / (nonfeb_means.mean() + eps))

    raw_mult = raw_mult.where(feb_counts >= min_feb_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply month adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


def predict_one_step(a_hist, n_lags, alpha):
    """Predict next-step values for all sectors."""
    return pd.Series({
        sector: ewgm_per_sector(a_hist, sector, n_lags, alpha)
        if a_hist[sector].tail(n_lags).min() > 0 else 0.0
        for sector in a_hist.columns
    })


def predict_horizon(a_tr, n_lags, alpha, t2):
    """Forecast horizon [67..78]."""
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)

    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            preds[sector] = ewgm_per_sector(a_tr, sector, n_lags, alpha)

    preds.index.name = 'time'
    return preds


def validate_ewgm_mape(a_tr, n_lags=6, alpha=0.5, t2=6, clip_low=0.85, clip_high=1.4, val_len=6):
    """Validate EWGM model on last val_len months and return average MAPE."""
    times = a_tr.index
    if len(times) < max(n_lags + 1, t2 + 1) + val_len:
        print("Not enough data for validation")
        return None
    
    mapes = []
    for t in times[-val_len:]:
        a_hist = a_tr.loc[a_tr.index < t]
        if len(a_hist) < max(n_lags, t2):
            continue
        
        y_true = a_tr.loc[t]
        y_pred = predict_one_step(a_hist, n_lags, alpha)
        
        if t % 12 == 11:
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)
        
        mask = y_true > 0
        ape = np.abs((y_pred[mask] - y_true[mask]) / y_true[mask])
        mape = ape.mean() * 100
        mapes.append(mape)
        
        print(f"Month {t}: MAPE = {mape:.2f}%")
    
    avg_mape = np.mean(mapes)
    print(f"\nAverage MAPE: {avg_mape:.2f}%")
    return avg_mape


# =====================================================
# Submission
# =====================================================

def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)
    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


def generate_ensemble_submission_with_february():
    """Ensemble with both December and February adjustments."""
    month_codes = build_month_codes()
    train, test = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    
    print("Generating ensemble predictions...")
    
    pred1 = predict_horizon(a_tr, n_lags=6, alpha=0.5, t2=6)
    pred2 = predict_horizon(a_tr, n_lags=9, alpha=0.67, t2=4)
    pred3 = predict_horizon(a_tr, n_lags=12, alpha=0.4, t2=8)
    
    a_pred = (pred1 + pred2 + pred3) / 3
    print("Ensemble averaged")
    
    # Apply December bump
    dec_mult = compute_december_multipliers(a_tr, clip_low=0.85, clip_high=1.4)
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], dec_mult)
    
    # Apply February bump
    feb_mult = compute_february_multipliers(a_tr, clip_low=0.85, clip_high=1.4)
    for t in a_pred.index[a_pred.index % 12 == 1]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], feb_mult)
    
    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('/Users/nikola/Python/KaggleCompetition/output/February_Bump/submission_feb.csv', index=False)
    print("Ensemble with February adjustment saved")
    return sub


# =====================================================
# Main
# =====================================================

def main():
    month_codes = build_month_codes()
    train, _ = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    
    # Validate baseline
    print("="*60)
    print("BASELINE VALIDATION")
    print("="*60)
    baseline_mape = validate_ewgm_mape(a_tr, n_lags=6, alpha=0.5, t2=6,
                                       clip_low=0.85, clip_high=1.4, val_len=6)
    
    # Generate February ensemble submission
    print("\n" + "="*60)
    print("GENERATING ENSEMBLE WITH FEBRUARY ADJUSTMENT")
    print("="*60)
    generate_ensemble_submission_with_february()


if __name__ == "__main__":
    main()

BASELINE VALIDATION
Month 61: MAPE = 168.90%
Month 62: MAPE = 49.99%
Month 63: MAPE = 62.78%
Month 64: MAPE = 49.14%
Month 65: MAPE = 46.38%
Month 66: MAPE = 75.13%

Average MAPE: 75.39%

GENERATING ENSEMBLE WITH FEBRUARY ADJUSTMENT
Generating ensemble predictions...
Ensemble averaged
Ensemble with February adjustment saved


# All methods combined - Score: 0.55889

In [1]:
import pandas as pd
import numpy as np
from functools import partial
import optuna

# =====================================================
# Utility Functions
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:  # test data
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:  # train data
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data():
    """Load competition training and test datasets."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/new_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    return train, test


# =====================================================
# Data Transformation
# =====================================================

def build_amount_matrix(train: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Pivot training data into [time x sector_id] transaction matrix."""
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_new_house_transactions', fill_value=0
    )

    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)
    return pivot


# =====================================================
# Modeling Helpers
# =====================================================

def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def compute_february_multipliers(a_tr, eps=1e-9, min_feb_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level February multipliers (Chinese New Year effect)."""
    is_feb = (a_tr.index % 12 == 1)
    feb_means = a_tr[is_feb].mean()
    nonfeb_means = a_tr[~is_feb].mean()
    feb_counts = a_tr[is_feb].count()

    raw_mult = feb_means / (nonfeb_means + eps)
    overall_mult = float(feb_means.mean() / (nonfeb_means.mean() + eps))

    raw_mult = raw_mult.where(feb_counts >= min_feb_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply month adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


def predict_one_step(a_hist, n_lags, alpha):
    """Predict next-step values for all sectors."""
    return pd.Series({
        sector: ewgm_per_sector(a_hist, sector, n_lags, alpha)
        if a_hist[sector].tail(n_lags).min() > 0 else 0.0
        for sector in a_hist.columns
    })


def predict_horizon(a_tr, n_lags, alpha, t2):
    """Forecast horizon [67..78]."""
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)

    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            preds[sector] = ewgm_per_sector(a_tr, sector, n_lags, alpha)

    preds.index.name = 'time'
    return preds


def evaluate_params(a_tr_full, n_lags, alpha, t2, clip_low, clip_high, val_len=6):
    """Evaluate parameters via rolling-origin backtest."""
    times = a_tr_full.index
    if len(times) < max(n_lags + 1, t2 + 1) + val_len:
        return 1e12

    rmses = []
    for t in times[-val_len:]:
        a_hist = a_tr_full.loc[a_tr_full.index < t]
        if len(a_hist) < max(n_lags, t2):
            continue

        y_true = a_tr_full.loc[t]
        y_pred = predict_one_step(a_hist, n_lags, alpha)

        if t % 12 == 11:
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)

        rmses.append(np.sqrt(np.mean((y_pred - y_true) ** 2)))

    return float(np.mean(rmses)) if rmses else 1e12


# =====================================================
# Submission
# =====================================================

def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)
    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


# =====================================================
# Optuna Optimization
# =====================================================

def optuna_objective(trial, a_tr):
    """Objective for Optuna hyperparameter tuning."""
    n_lags = trial.suggest_int('n_lags', 3, 12)
    alpha = trial.suggest_float('alpha', 0.20, 0.95)
    t2 = trial.suggest_int('t2', 3, 9)
    clip_low = trial.suggest_float('clip_low', 0.70, 0.95)
    clip_high = trial.suggest_float('clip_high', 1.10, 1.80)

    if clip_low >= clip_high:
        clip_low = max(0.70, clip_high - 0.05)

    return evaluate_params(a_tr, n_lags, alpha, t2, clip_low, clip_high)


def run_optuna_search(a_tr, n_trials=512, seed=1337):
    """Run Optuna search and return the study."""
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(partial(optuna_objective, a_tr=a_tr), n_trials=n_trials, show_progress_bar=False)
    return study


# =====================================================
# Main
# =====================================================

def main():
    from sklearn.preprocessing import StandardScaler
    
    month_codes = build_month_codes()
    train, test = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    
    # Step 1: Run Optuna optimization
    print("="*60)
    print("RUNNING OPTUNA HYPERPARAMETER OPTIMIZATION (512 trials)")
    print("="*60)
    study = run_optuna_search(a_tr, n_trials=512, seed=1337)
    best = study.best_params
    print(f"\nBest parameters: {best}")
    print(f"Best RMSE: {study.best_value:.2f}")
    
    # Step 2: Generate combined submission with optimized parameters
    print("\n" + "="*60)
    print("GENERATING COMBINED SUBMISSION (POI + ENSEMBLE + FEBRUARY)")
    print("="*60)
    print("Using optimized parameters in ensemble...")
    
    # Load POI features
    poi = pd.read_csv('/Users/nikola/Python/KaggleCompetition/data/train/sector_poi.csv')
    poi_cols = ['office_population', 'education', 'surrounding_housing_average_price', 
                'medical_health', 'subway_station_cnt']
    poi['sector_id'] = poi['sector'].str.extract(r'(\d+)').astype(int)
    poi_features = poi[['sector_id'] + poi_cols].set_index('sector_id')
    
    scaler = StandardScaler()
    poi_features[poi_cols] = scaler.fit_transform(poi_features[poi_cols])
    all_sectors = np.arange(1, 97)
    poi_features = poi_features.reindex(all_sectors, fill_value=0)
    
    print("Generating ensemble predictions with POI...")
    
    # Generate 3 predictions using baseline + optimized + conservative
    pred1 = predict_horizon(a_tr, n_lags=6, alpha=0.5, t2=6)  # Baseline
    pred2 = predict_horizon(a_tr, n_lags=best['n_lags'], alpha=best['alpha'], t2=best['t2'])  # Optuna best
    pred3 = predict_horizon(a_tr, n_lags=12, alpha=0.4, t2=8)  # Conservative
    
    # Apply POI adjustment
    poi_weight = 0.05
    for sector in pred1.columns:
        poi_score = poi_features.loc[sector].sum()
        poi_mult = np.clip(1.0 + (poi_score * poi_weight), 0.85, 1.15)
        pred1[sector] *= poi_mult
        pred2[sector] *= poi_mult
        pred3[sector] *= poi_mult
    
    a_pred = (pred1 + pred2 + pred3) / 3
    print("Ensemble with POI complete")
    
    # Apply December bump
    dec_mult = compute_december_multipliers(a_tr, 
                                           clip_low=best['clip_low'], 
                                           clip_high=best['clip_high'])
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], dec_mult)
    
    # Apply February bump
    feb_mult = compute_february_multipliers(a_tr, 
                                           clip_low=best['clip_low'], 
                                           clip_high=best['clip_high'])
    for t in a_pred.index[a_pred.index % 12 == 1]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], feb_mult)
    
    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('/Users/nikola/Python/KaggleCompetition/output/EWGM_COMBINED/submission_combined_optuna.csv', index=False)
    print("\nCombined submission with Optuna optimization saved!")
    print("File: submission_combined_optuna.csv")


if __name__ == "__main__":
    main()

[I 2025-09-30 12:16:37,264] A new study created in memory with name: no-name-5c0ced1f-a292-4a93-a17c-8a6c670671a6
[I 2025-09-30 12:16:37,281] Trial 0 finished with value: 21383.972104866807 and parameters: {'n_lags': 5, 'alpha': 0.31901297911584925, 't2': 4, 'clip_low': 0.8148292218036416, 'clip_high': 1.3247003783641171}. Best is trial 0 with value: 21383.972104866807.
[I 2025-09-30 12:16:37,297] Trial 1 finished with value: 21231.50264119316 and parameters: {'n_lags': 8, 'alpha': 0.3964571941738585, 't2': 9, 'clip_low': 0.8832036381726205, 'clip_high': 1.1806919586782048}. Best is trial 1 with value: 21231.50264119316.
[I 2025-09-30 12:16:37,313] Trial 2 finished with value: 21112.848609130506 and parameters: {'n_lags': 6, 'alpha': 0.6713758846547837, 't2': 3, 'clip_low': 0.9458871512859103, 'clip_high': 1.4102574080515897}. Best is trial 2 with value: 21112.848609130506.
[I 2025-09-30 12:16:37,329] Trial 3 finished with value: 21122.422935786373 and parameters: {'n_lags': 10, 'alpha

RUNNING OPTUNA HYPERPARAMETER OPTIMIZATION (512 trials)


[I 2025-09-30 12:16:37,498] Trial 11 finished with value: 21094.005759207124 and parameters: {'n_lags': 7, 'alpha': 0.6136437470111018, 't2': 7, 'clip_low': 0.9480450968302885, 'clip_high': 1.5953497173720117}. Best is trial 10 with value: 21093.690894088948.
[I 2025-09-30 12:16:37,527] Trial 12 finished with value: 21096.721246042234 and parameters: {'n_lags': 7, 'alpha': 0.5926902717110654, 't2': 7, 'clip_low': 0.9442375014368745, 'clip_high': 1.6138738076887598}. Best is trial 10 with value: 21093.690894088948.
[I 2025-09-30 12:16:37,554] Trial 13 finished with value: 21700.647970117716 and parameters: {'n_lags': 8, 'alpha': 0.20563828697295955, 't2': 7, 'clip_low': 0.9050039798962632, 'clip_high': 1.6018220250780288}. Best is trial 10 with value: 21093.690894088948.
[I 2025-09-30 12:16:37,591] Trial 14 finished with value: 21080.031173804742 and parameters: {'n_lags': 8, 'alpha': 0.5998900635155954, 't2': 8, 'clip_low': 0.9115106298560189, 'clip_high': 1.7977978030395279}. Best is 


Best parameters: {'n_lags': 9, 'alpha': 0.6741577959564671, 't2': 4, 'clip_low': 0.8798137129337238, 'clip_high': 1.205522354649888}
Best RMSE: 21046.09

GENERATING COMBINED SUBMISSION (POI + ENSEMBLE + FEBRUARY)
Using optimized parameters in ensemble...
Generating ensemble predictions with POI...
Ensemble with POI complete

Combined submission with Optuna optimization saved!
File: submission_combined_optuna.csv


# Weighted ensemble approach: 50% POI 30% Baseline and 20% Conservative - Score: 0.55980

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# =====================================================
# Utility Functions
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:  # test data
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:  # train data
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data():
    """Load competition training and test datasets."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/new_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    return train, test


def build_amount_matrix(train: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Pivot training data into [time x sector_id] transaction matrix."""
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_new_house_transactions', fill_value=0
    )

    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)
    return pivot


def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply month adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


def predict_horizon(a_tr, n_lags, alpha, t2):
    """Forecast horizon [67..78]."""
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)

    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            preds[sector] = ewgm_per_sector(a_tr, sector, n_lags, alpha)

    preds.index.name = 'time'
    return preds


def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)
    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


def generate_weighted_ensemble():
    """Weighted ensemble: 60% baseline, 25% aggressive, 15% conservative."""
    month_codes = build_month_codes()
    train, test = load_competition_data()
    a_tr = build_amount_matrix(train, month_codes)
    
    # Load POI features
    poi = pd.read_csv('/Users/nikola/Python/KaggleCompetition/data/train/sector_poi.csv')
    poi_cols = ['office_population', 'education', 'surrounding_housing_average_price', 
                'medical_health', 'subway_station_cnt']
    poi['sector_id'] = poi['sector'].str.extract(r'(\d+)').astype(int)
    poi_features = poi[['sector_id'] + poi_cols].set_index('sector_id')
    
    scaler = StandardScaler()
    poi_features[poi_cols] = scaler.fit_transform(poi_features[poi_cols])
    all_sectors = np.arange(1, 97)
    poi_features = poi_features.reindex(all_sectors, fill_value=0)
    
    print("Generating weighted ensemble...")
    
    # Model 1: Baseline (highest weight since POI scored best)
    pred1 = predict_horizon(a_tr, n_lags=6, alpha=0.5, t2=6)
    poi_weight1 = 0.08  # Slightly higher POI weight for best model
    for sector in pred1.columns:
        poi_score = poi_features.loc[sector].sum()
        poi_mult = np.clip(1.0 + (poi_score * poi_weight1), 0.85, 1.15)
        pred1[sector] *= poi_mult
    
    # Model 2: Aggressive
    pred2 = predict_horizon(a_tr, n_lags=9, alpha=0.67, t2=4)
    poi_weight2 = 0.03
    for sector in pred2.columns:
        poi_score = poi_features.loc[sector].sum()
        poi_mult = np.clip(1.0 + (poi_score * poi_weight2), 0.9, 1.1)
        pred2[sector] *= poi_mult
    
    # Model 3: Conservative
    pred3 = predict_horizon(a_tr, n_lags=12, alpha=0.4, t2=8)
    
    # Weighted average: 60% baseline, 25% aggressive, 15% conservative
    a_pred = (pred1 * 0.60) + (pred2 * 0.25) + (pred3 * 0.15)
    print("Weighted ensemble: 60% baseline + 25% aggressive + 15% conservative")
    
    # Apply December bump
    dec_mult = compute_december_multipliers(a_tr, clip_low=0.85, clip_high=1.35)
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], dec_mult)
    
    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('/Users/nikola/Python/KaggleCompetition/output/weighted_Ensemble/submission_weighted.csv', index=False)
    print("Weighted ensemble submission saved")
    return sub


def main():
    print("="*60)
    print("GENERATING WEIGHTED ENSEMBLE")
    print("="*60)
    generate_weighted_ensemble()


if __name__ == "__main__":
    main()

GENERATING WEIGHTED ENSEMBLE
Generating weighted ensemble...
Weighted ensemble: 60% baseline + 25% aggressive + 15% conservative
Weighted ensemble submission saved


# Visualisation of POI highest score

In [5]:
import pandas as pd
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns

# =====================================================
# Utility Functions
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:  # test data
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:  # train data
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data_with_poi():
    """Load competition data including POI features."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/new_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    poi = pd.read_csv(f'{path}/train/sector_poi.csv')
    return train, test, poi


# =====================================================
# Data Transformation
# =====================================================

def build_amount_matrix_with_poi(train: pd.DataFrame, poi: pd.DataFrame, month_codes: dict):
    """Build transaction matrix and POI features."""
    from sklearn.preprocessing import StandardScaler
    
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_new_house_transactions', fill_value=0
    )
    
    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)
    
    poi_cols = ['office_population', 'education', 'surrounding_housing_average_price', 
                'medical_health', 'subway_station_cnt', 'number_of_supermarket_convenience_stores', 'shopping_centers_dense']
    
    poi['sector_id'] = poi['sector'].str.extract(r'(\d+)').astype(int)
    poi_features = poi[['sector_id'] + poi_cols].set_index('sector_id')
    
    scaler = StandardScaler()
    poi_features[poi_cols] = scaler.fit_transform(poi_features[poi_cols])
    poi_features = poi_features.reindex(all_sectors, fill_value=0)
    
    return pivot, poi_features


# =====================================================
# Modeling Helpers
# =====================================================

def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply December adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


def ewgm_per_sector_with_poi(a_tr, poi_features, sector, n_lags, alpha, poi_weight=0.1):
    """Enhanced EWGM with POI adjustment."""
    base_pred = ewgm_per_sector(a_tr, sector, n_lags, alpha)
    if base_pred == 0:
        return 0.0
    
    poi_score = poi_features.loc[sector].sum()
    poi_multiplier = 1.0 + (poi_score * poi_weight)
    poi_multiplier = np.clip(poi_multiplier, 0.7, 1.3)
    
    return base_pred * poi_multiplier


def predict_horizon_with_poi(a_tr, poi_features, n_lags, alpha, t2, poi_weight=0.1):
    """Forecast with POI-enhanced predictions."""
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)
    
    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            preds[sector] = ewgm_per_sector_with_poi(a_tr, poi_features, sector, n_lags, alpha, poi_weight)
    
    preds.index.name = 'time'
    return preds


# =====================================================
# Visualization Functions
# =====================================================

def visualize_poi_impact(poi_features, poi_weight=0.1):
    """Visualize POI multiplier effect by sector."""
    poi_scores = poi_features.sum(axis=1)
    poi_multipliers = np.clip(1.0 + (poi_scores * poi_weight), 0.7, 1.3)
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Top sectors by POI boost
    top_sectors = poi_multipliers.nlargest(20)
    axes[0].barh(range(len(top_sectors)), top_sectors.values, color='green', alpha=0.7)
    axes[0].set_yticks(range(len(top_sectors)))
    axes[0].set_yticklabels([f'Sector {s}' for s in top_sectors.index])
    axes[0].axvline(x=1.0, color='red', linestyle='--', label='No adjustment')
    axes[0].set_xlabel('POI Multiplier')
    axes[0].set_title('Top 20 Sectors - POI Boost Effect')
    axes[0].legend()
    axes[0].invert_yaxis()
    
    # Distribution of POI multipliers
    axes[1].hist(poi_multipliers.values, bins=30, color='steelblue', alpha=0.7, edgecolor='black')
    axes[1].axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='No adjustment')
    axes[1].set_xlabel('POI Multiplier')
    axes[1].set_ylabel('Number of Sectors')
    axes[1].set_title('Distribution of POI Multipliers Across All Sectors')
    axes[1].legend()
    
    plt.tight_layout()
    plt.savefig('/Users/nikola/Python/KaggleCompetition/output/visualizations/poi_impact.png', dpi=300, bbox_inches='tight')
    print("Saved: poi_impact.png")
    plt.close()


def visualize_december_multipliers(a_tr):
    """Visualize December seasonal adjustment by sector."""
    dec_mult = compute_december_multipliers(a_tr, clip_low=0.85, clip_high=1.4)
    mult_series = pd.Series(dec_mult).sort_values(ascending=False)
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Top sectors by December boost
    top_20 = mult_series.head(20)
    axes[0].barh(range(len(top_20)), top_20.values, color='darkred', alpha=0.7)
    axes[0].set_yticks(range(len(top_20)))
    axes[0].set_yticklabels([f'Sector {s}' for s in top_20.index])
    axes[0].axvline(x=1.0, color='blue', linestyle='--', label='No adjustment')
    axes[0].set_xlabel('December Multiplier')
    axes[0].set_title('Top 20 Sectors - December Boost Effect')
    axes[0].legend()
    axes[0].invert_yaxis()
    
    # Distribution of December multipliers
    axes[1].hist(mult_series.values, bins=30, color='coral', alpha=0.7, edgecolor='black')
    axes[1].axvline(x=1.0, color='blue', linestyle='--', linewidth=2, label='No adjustment')
    axes[1].set_xlabel('December Multiplier')
    axes[1].set_ylabel('Number of Sectors')
    axes[1].set_title('Distribution of December Multipliers Across All Sectors')
    axes[1].legend()
    
    plt.tight_layout()
    plt.savefig('/Users/nikola/Python/KaggleCompetition/output/visualizations/december_multipliers.png', dpi=300, bbox_inches='tight')
    print("Saved: december_multipliers.png")
    plt.close()


def visualize_validation_performance(a_tr, poi_features, n_lags=6, alpha=0.5, t2=6, 
                                     clip_low=0.85, clip_high=1.4, poi_weight=0.1, val_len=6):
    """Visualize prediction vs actual for validation period."""
    times = a_tr.index[-val_len:]
    predictions = []
    actuals = []
    
    for t in times:
        a_hist = a_tr.loc[a_tr.index < t]
        y_true = a_tr.loc[t]
        
        y_pred = pd.Series({
            sector: ewgm_per_sector_with_poi(a_hist, poi_features, sector, n_lags, alpha, poi_weight)
            if a_hist[sector].tail(n_lags).min() > 0 else 0.0
            for sector in a_hist.columns
        })
        
        if t % 12 == 11:
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)
        
        predictions.append(y_pred.sum())
        actuals.append(y_true.sum())
    
    fig, ax = plt.subplots(figsize=(12, 6))
    x = range(len(times))
    
    ax.plot(x, actuals, marker='o', linewidth=2, label='Actual', color='blue')
    ax.plot(x, predictions, marker='s', linewidth=2, label='Predicted', color='red')
    ax.fill_between(x, actuals, predictions, alpha=0.2, color='gray')
    
    ax.set_xlabel('Validation Month')
    ax.set_ylabel('Total Transactions (All Sectors)')
    ax.set_title('Validation Performance: Predicted vs Actual')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xticks(x)
    ax.set_xticklabels([f'Month {t}' for t in times])
    
    plt.tight_layout()
    plt.savefig('/Users/nikola/Python/KaggleCompetition/output/visualizations/validation_performance.png', dpi=300, bbox_inches='tight')
    print("Saved: validation_performance.png")
    plt.close()


# =====================================================
# Submission
# =====================================================

def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)
    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


def generate_submission_poi_enhanced(n_lags=6, alpha=0.5, t2=6, clip_low=0.85, clip_high=1.4, poi_weight=0.1):
    """End-to-end pipeline with POI features."""
    month_codes = build_month_codes()
    train, test, poi = load_competition_data_with_poi()
    
    a_tr, poi_features = build_amount_matrix_with_poi(train, poi, month_codes)
    a_pred = predict_horizon_with_poi(a_tr, poi_features, n_lags, alpha, t2, poi_weight)
    
    mult = compute_december_multipliers(a_tr, clip_low=clip_low, clip_high=clip_high)
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], mult)
    
    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('/Users/nikola/Python/KaggleCompetition/output/EWGM_POI/EWGM_POI_submission_poi.csv', index=False)
    print(f"POI-enhanced submission saved (poi_weight={poi_weight})")
    return sub


# =====================================================
# Main
# =====================================================

def main():
    import os
    os.makedirs('/Users/nikola/Python/KaggleCompetition/output/visualizations', exist_ok=True)
    
    month_codes = build_month_codes()
    train, test, poi = load_competition_data_with_poi()
    a_tr, poi_features = build_amount_matrix_with_poi(train, poi, month_codes)
    
    print("="*60)
    print("GENERATING VISUALIZATIONS")
    print("="*60)
    
    # Generate visualizations
    visualize_poi_impact(poi_features, poi_weight=0.1)
    visualize_december_multipliers(a_tr)
    visualize_validation_performance(a_tr, poi_features, 
                                     n_lags=6, alpha=0.5, t2=6,
                                     clip_low=0.85, clip_high=1.4, 
                                     poi_weight=0.1, val_len=6)
    
    print("\n" + "="*60)
    print("GENERATING SUBMISSION")
    print("="*60)
    generate_submission_poi_enhanced(
        n_lags=6, alpha=0.5, t2=6, 
        clip_low=0.85, clip_high=1.4, 
        poi_weight=0.1
    )
    print("\nAll visualizations saved to: output/visualizations/")


if __name__ == "__main__":
    main()

GENERATING VISUALIZATIONS
Saved: poi_impact.png
Saved: december_multipliers.png
Saved: validation_performance.png

GENERATING SUBMISSION
POI-enhanced submission saved (poi_weight=0.1)

All visualizations saved to: output/visualizations/


# Optimized Alpha: POI EWGM - Score: probably lower

In [4]:
import pandas as pd
import numpy as np
from functools import partial

# =====================================================
# Utility Functions
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:  # test data
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:  # train data
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data_with_poi():
    """Load competition data including POI features."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/pre_owned_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    poi = pd.read_csv(f'{path}/train/sector_POI.csv')
    return train, test, poi


# =====================================================
# Data Transformation
# =====================================================

def build_amount_matrix_with_poi(train: pd.DataFrame, poi: pd.DataFrame, month_codes: dict):
    """Build transaction matrix and POI features."""
    from sklearn.preprocessing import StandardScaler
    
    # Build transaction matrix
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_pre_owned_house_transactions', fill_value=0
    )
    
    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)
    
    # Prepare POI features - use actual column names from your CSV
    poi_cols = ['office_population', 'education', 'surrounding_housing_average_price', 
                'medical_health', 'subway_station_cnt']
    
    poi['sector_id'] = poi['sector'].str.extract(r'(\d+)').astype(int)
    poi_features = poi[['sector_id'] + poi_cols].set_index('sector_id')
    
    # Normalize POI features
    scaler = StandardScaler()
    poi_features[poi_cols] = scaler.fit_transform(poi_features[poi_cols])
    poi_features = poi_features.reindex(all_sectors, fill_value=0)
    
    return pivot, poi_features

# =====================================================
# Modeling Helpers
# =====================================================

def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply December adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


# =====================================================
# ALPHA OPTIMIZATION FUNCTIONS (NEW)
# =====================================================

def find_best_alpha_for_sector(a_tr, sector, n_lags=6, val_months=6, 
                               alpha_candidates=[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    """
    Find the best alpha for a single sector using validation.
    Returns best_alpha and its validation MAPE.
    """
    # Check if sector has enough non-zero data
    if a_tr[sector].sum() == 0 or len(a_tr) < n_lags + val_months:
        return 0.5, None  # Default alpha
    
    best_alpha = 0.5
    best_mape = float('inf')
    
    # Test each alpha candidate
    for alpha in alpha_candidates:
        mapes = []
        
        # Walk-forward validation on last val_months
        for t in a_tr.index[-val_months:]:
            a_hist = a_tr.loc[a_tr.index < t]
            if len(a_hist) < n_lags:
                continue
            
            # Predict
            pred = ewgm_per_sector(a_hist, sector, n_lags, alpha)
            actual = a_tr.loc[t, sector]
            
            # Calculate APE (absolute percentage error)
            if actual > 0:
                ape = abs((pred - actual) / actual)
                mapes.append(ape)
        
        # Calculate average MAPE for this alpha
        if mapes:
            avg_mape = np.mean(mapes) * 100
            if avg_mape < best_mape:
                best_mape = avg_mape
                best_alpha = alpha
    
    return best_alpha, best_mape


def optimize_alphas_for_all_sectors(a_tr, n_lags=6, val_months=6):
    """
    Find optimal alpha for each sector.
    Returns dictionary: {sector_id: best_alpha}
    """
    print("="*60)
    print("OPTIMIZING ALPHA FOR EACH SECTOR")
    print("="*60)
    
    sector_alphas = {}
    sector_mapes = {}
    
    for i, sector in enumerate(a_tr.columns, 1):
        if i % 10 == 0:
            print(f"Progress: {i}/{len(a_tr.columns)} sectors")
        
        best_alpha, best_mape = find_best_alpha_for_sector(
            a_tr, sector, n_lags, val_months
        )
        sector_alphas[sector] = best_alpha
        sector_mapes[sector] = best_mape
    
    # Print summary statistics
    alpha_values = list(sector_alphas.values())
    print(f"\nAlpha Distribution:")
    print(f"  Mean: {np.mean(alpha_values):.2f}")
    print(f"  Median: {np.median(alpha_values):.2f}")
    print(f"  Min: {np.min(alpha_values):.2f}")
    print(f"  Max: {np.max(alpha_values):.2f}")
    
    # Show how many sectors prefer each alpha
    from collections import Counter
    alpha_counts = Counter(alpha_values)
    print(f"\nAlpha Usage:")
    for alpha in sorted(alpha_counts.keys()):
        print(f"  α={alpha}: {alpha_counts[alpha]} sectors")
    
    return sector_alphas, sector_mapes


# =====================================================
# Prediction Functions (MODIFIED)
# =====================================================

def predict_horizon_with_sector_alphas(a_tr, sector_alphas, n_lags, t2):
    """Forecast with sector-specific alphas."""
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)
    
    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            # Use sector-specific alpha
            alpha = sector_alphas.get(sector, 0.5)
            preds[sector] = ewgm_per_sector(a_tr, sector, n_lags, alpha)
    
    preds.index.name = 'time'
    return preds


# =====================================================
# Submission
# =====================================================

def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)
    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


def generate_submission_optimized(sector_alphas, n_lags=6, t2=6, clip_low=0.85, clip_high=1.4):
    """End-to-end pipeline with optimized alphas."""
    month_codes = build_month_codes()
    train, test, poi = load_competition_data_with_poi()
    
    a_tr, poi_features = build_amount_matrix_with_poi(train, poi, month_codes)
    a_pred = predict_horizon_with_sector_alphas(a_tr, sector_alphas, n_lags, t2)
    
    # Apply December bump
    mult = compute_december_multipliers(a_tr, clip_low=clip_low, clip_high=clip_high)
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], mult)
    
    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('ewgm_optimized_alphas.csv', index=False)
    print(f"Optimized submission saved")
    return sub


# =====================================================
# Main
# =====================================================

def main():
    month_codes = build_month_codes()
    train, test, poi = load_competition_data_with_poi()
    a_tr, poi_features = build_amount_matrix_with_poi(train, poi, month_codes)
    
    print(f"Data loaded: {len(a_tr)} months, {len(a_tr.columns)} sectors")
    
    # STEP 1: Optimize alphas for all sectors
    sector_alphas, sector_mapes = optimize_alphas_for_all_sectors(
        a_tr, n_lags=6, val_months=6
    )
    
    # Save alpha mappings
    alpha_df = pd.DataFrame({
        'sector': list(sector_alphas.keys()),
        'best_alpha': list(sector_alphas.values()),
        'validation_mape': [sector_mapes.get(s) for s in sector_alphas.keys()]
    })
    alpha_df.to_csv('sector_alphas.csv', index=False)
    print("\nSector alphas saved to sector_alphas.csv")
    
    # STEP 2: Validate with optimized alphas
    print("\n" + "="*60)
    print("VALIDATION WITH OPTIMIZED ALPHAS")
    print("="*60)
    
    mapes = []
    for t in a_tr.index[-6:]:
        a_hist = a_tr.loc[a_tr.index < t]
        if len(a_hist) < 6:
            continue
        
        y_true = a_tr.loc[t]
        y_pred = pd.Series({
            sector: ewgm_per_sector(a_hist, sector, 6, sector_alphas.get(sector, 0.5))
            if a_hist[sector].tail(6).min() > 0 else 0.0
            for sector in a_hist.columns
        })
        
        # Apply December bump if December
        if t % 12 == 11:
            mult = compute_december_multipliers(a_hist, clip_low=0.85, clip_high=1.4)
            y_pred = apply_december_bump_row(y_pred, mult)
        
        # Calculate MAPE
        mask = y_true > 0
        ape = np.abs((y_pred[mask] - y_true[mask]) / y_true[mask])
        mape = ape.mean() * 100
        mapes.append(mape)
        print(f"Month {t}: MAPE = {mape:.2f}%")
    
    avg_mape = np.mean(mapes)
    print(f"\nAverage MAPE: {avg_mape:.2f}%")
    print(f"Competition score estimate: {1 - (avg_mape/100):.5f}")
    
    # STEP 3: Generate submission
    print("\n" + "="*60)
    print("GENERATING SUBMISSION WITH OPTIMIZED ALPHAS")
    print("="*60)
    
    sub = generate_submission_optimized(
        sector_alphas,
        n_lags=6, t2=6, 
        clip_low=0.85, clip_high=1.4
    )
    
    print(f"\nSubmission saved: ewgm_optimized_alphas.csv")
    print(f"Prediction stats:")
    print(f"  Total rows: {len(sub)}")
    print(f"  Mean: {sub['new_house_transaction_amount'].mean():.2f}")
    print(f"  Median: {sub['new_house_transaction_amount'].median():.2f}")
    print(f"  Min: {sub['new_house_transaction_amount'].min():.2f}")
    print(f"  Max: {sub['new_house_transaction_amount'].max():.2f}")


if __name__ == "__main__":
    main()

Data loaded: 67 months, 96 sectors
OPTIMIZING ALPHA FOR EACH SECTOR
Progress: 10/96 sectors
Progress: 20/96 sectors
Progress: 30/96 sectors
Progress: 40/96 sectors
Progress: 50/96 sectors
Progress: 60/96 sectors
Progress: 70/96 sectors
Progress: 80/96 sectors
Progress: 90/96 sectors

Alpha Distribution:
  Mean: 0.65
  Median: 0.70
  Min: 0.30
  Max: 0.90

Alpha Usage:
  α=0.3: 20 sectors
  α=0.4: 2 sectors
  α=0.5: 19 sectors
  α=0.6: 3 sectors
  α=0.7: 8 sectors
  α=0.8: 5 sectors
  α=0.9: 39 sectors

Sector alphas saved to sector_alphas.csv

VALIDATION WITH OPTIMIZED ALPHAS
Month 61: MAPE = 116.96%
Month 62: MAPE = 27.25%
Month 63: MAPE = 19.47%
Month 64: MAPE = 24.34%
Month 65: MAPE = 24.18%
Month 66: MAPE = 18.72%

Average MAPE: 38.49%
Competition score estimate: 0.61515

GENERATING SUBMISSION WITH OPTIMIZED ALPHAS
Optimized submission saved

Submission saved: ewgm_optimized_alphas.csv
Prediction stats:
  Total rows: 1152
  Mean: 14842.80
  Median: 8888.47
  Min: 0.00
  Max: 108165

# EWGM Optuna Hyperparameter Optimization  - Score:

In [10]:
import pandas as pd
import numpy as np
import optuna
from functools import partial

# =====================================================
# Utility Functions (Same as before)
# =====================================================

def build_month_codes():
    """Create a mapping from month abbreviations to numeric values."""
    return {m: i for i, m in enumerate(
        ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], start=1)}


def split_test_id_column(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the ID column into month text and sector components."""
    parts = df['id'].str.split('_', expand=True)
    df['month_text'], df['sector'] = parts[0], parts[1]
    return df


def add_time_and_sector_fields(df: pd.DataFrame, month_codes: dict) -> pd.DataFrame:
    """Add parsed year, month, time index, and sector_id to dataframe."""
    if 'sector' in df.columns:
        df['sector_id'] = df['sector'].str[7:].astype(int)

    if 'month' in df.columns:
        df['year'] = df['month'].str[:4].astype(int)
        df['month'] = df['month'].str[5:].map(month_codes)
    else:
        df['year'] = df['month_text'].str[:4].astype(int)
        df['month'] = df['month_text'].str[5:].map(month_codes)

    df['time'] = (df['year'] - 2019) * 12 + df['month'] - 1
    return df


def load_competition_data_with_poi():
    """Load competition data including POI features."""
    path = '/Users/nikola/Python/KaggleCompetition/data'
    train = pd.read_csv(f'{path}/train/pre_owned_house_transactions.csv')
    test = pd.read_csv(f'{path}/test.csv')
    poi = pd.read_csv(f'{path}/train/sector_POI.csv')
    return train, test, poi


def build_amount_matrix_with_poi(train: pd.DataFrame, poi: pd.DataFrame, month_codes: dict):
    """Build transaction matrix and POI features."""
    from sklearn.preprocessing import StandardScaler
    
    train = add_time_and_sector_fields(train.copy(), month_codes)
    pivot = train.pivot_table(
        index='time', columns='sector_id',
        values='amount_pre_owned_house_transactions', fill_value=0
    )
    
    all_sectors = np.arange(1, 97)
    pivot = pivot.reindex(columns=all_sectors, fill_value=0)
    
    poi_cols = ['office_population', 'education', 'surrounding_housing_average_price', 
                'medical_health', 'subway_station_cnt']
    
    poi['sector_id'] = poi['sector'].str.extract(r'(\d+)').astype(int)
    poi_features = poi[['sector_id'] + poi_cols].set_index('sector_id')
    
    scaler = StandardScaler()
    poi_features[poi_cols] = scaler.fit_transform(poi_features[poi_cols])
    poi_features = poi_features.reindex(all_sectors, fill_value=0)
    
    return pivot, poi_features


# =====================================================
# Core EWGM Functions
# =====================================================

def compute_december_multipliers(a_tr, eps=1e-9, min_dec_obs=1, clip_low=0.8, clip_high=1.5):
    """Compute sector-level December multipliers from training data."""
    is_dec = (a_tr.index % 12 == 11)
    dec_means = a_tr[is_dec].mean()
    nondec_means = a_tr[~is_dec].mean()
    dec_counts = a_tr[is_dec].count()

    raw_mult = dec_means / (nondec_means + eps)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + eps))

    raw_mult = raw_mult.where(dec_counts >= min_dec_obs, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    return raw_mult.clip(clip_low, clip_high).to_dict()


def apply_december_bump_row(pred_row: pd.Series, sector_to_mult: dict) -> pd.Series:
    """Apply December adjustment to a prediction row."""
    return pred_row.multiply(pd.Series(sector_to_mult)).fillna(pred_row)


def ewgm_per_sector(a_tr, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector."""
    recent = a_tr[sector].tail(n_lags).values
    if len(recent) < n_lags or (recent <= 0).all():
        return 0.0

    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)])
    weights /= weights.sum()

    mask = recent > 0
    if not mask.any():
        return 0.0

    log_vals = np.log(recent[mask] + 1e-12)
    pos_w = weights[mask] / weights[mask].sum()
    return float(np.exp(np.sum(pos_w * log_vals)))


# =====================================================
# Optuna Optimization
# =====================================================

def objective(trial, a_tr):
    """
    Optuna objective function to minimize MAPE.
    Tests different hyperparameters on validation set.
    """
    # Suggest hyperparameters
    alpha = trial.suggest_float('alpha', 0.3, 0.9)
    n_lags = trial.suggest_int('n_lags', 3, 12)
    clip_low = trial.suggest_float('clip_low', 0.7, 0.9)
    clip_high = trial.suggest_float('clip_high', 1.2, 1.8)
    t2 = trial.suggest_int('t2', 3, 12)  # Min non-zero values required
    
    # Validate on last 6 months
    val_months = 6
    mapes = []
    
    for t in a_tr.index[-val_months:]:
        a_hist = a_tr.loc[a_tr.index < t]
        if len(a_hist) < max(n_lags, t2):
            continue
        
        y_true = a_tr.loc[t]
        
        # Predict for all sectors
        y_pred = pd.Series({
            sector: ewgm_per_sector(a_hist, sector, n_lags, alpha)
            if a_hist[sector].tail(t2).min() > 0 else 0.0
            for sector in a_hist.columns
        })
        
        # Apply December bump if December
        if t % 12 == 11:
            mult = compute_december_multipliers(a_hist, clip_low=clip_low, clip_high=clip_high)
            y_pred = apply_december_bump_row(y_pred, mult)
        
        # Calculate MAPE
        mask = y_true > 0
        if mask.sum() == 0:
            continue
        
        ape = np.abs((y_pred[mask] - y_true[mask]) / y_true[mask])
        mape = ape.mean() * 100
        mapes.append(mape)
    
    if not mapes:
        return float('inf')
    
    avg_mape = np.mean(mapes)
    return avg_mape


def run_optuna_optimization(a_tr, n_trials=100):
    """
    Run Optuna hyperparameter optimization.
    """
    print("="*60)
    print(f"OPTUNA OPTIMIZATION ({n_trials} trials)")
    print("="*60)
    
    # Create study
    study = optuna.create_study(
        direction='minimize',
        sampler=optuna.samplers.TPESampler(seed=42)
    )
    
    # Optimize
    study.optimize(
        lambda trial: objective(trial, a_tr),
        n_trials=n_trials,
        show_progress_bar=True
    )
    
    # Results
    print("\n" + "="*60)
    print("OPTIMIZATION RESULTS")
    print("="*60)
    
    best_params = study.best_params
    best_mape = study.best_value
    best_score = 1 - (best_mape / 100)
    
    print(f"\nBest parameters:")
    for param, value in best_params.items():
        print(f"  {param}: {value}")
    
    print(f"\nBest validation MAPE: {best_mape:.2f}%")
    print(f"Competition score estimate: {best_score:.5f}")
    
    # Show top 5 trials
    print("\nTop 5 configurations:")
    trials_df = study.trials_dataframe()
    trials_df['score'] = 1 - (trials_df['value'] / 100)
    top5 = trials_df.nsmallest(5, 'value')[['number', 'value', 'score'] + 
                                           [c for c in trials_df.columns if c.startswith('params_')]]
    print(top5.to_string(index=False))
    
    return best_params, best_score


# =====================================================
# Prediction and Submission
# =====================================================

def predict_horizon_optimized(a_tr, params):
    """Forecast with optimized parameters."""
    alpha = params['alpha']
    n_lags = params['n_lags']
    t2 = params['t2']
    
    idx = np.arange(67, 79)
    preds = pd.DataFrame(index=idx, columns=a_tr.columns, dtype=float)
    
    for sector in a_tr.columns:
        if (a_tr[sector].tail(t2).min() == 0) or (a_tr[sector].sum() == 0):
            preds[sector] = 0.0
        else:
            preds[sector] = ewgm_per_sector(a_tr, sector, n_lags, alpha)
    
    preds.index.name = 'time'
    return preds


def build_submission_df(a_pred, test_raw, month_codes):
    """Format predictions into competition submission file."""
    test = add_time_and_sector_fields(split_test_id_column(test_raw.copy()), month_codes)
    lookup = a_pred.stack().rename('pred').reset_index().rename(columns={'level_1': 'sector_id'})
    merged = test.merge(lookup, on=['time', 'sector_id'], how='left')
    merged['pred'] = merged['pred'].fillna(0.0)
    return merged[['id', 'pred']].rename(columns={'pred': 'new_house_transaction_amount'})


def generate_submission_optimized(best_params):
    """Generate submission with optimized parameters."""
    month_codes = build_month_codes()
    train, test, poi = load_competition_data_with_poi()
    
    a_tr, poi_features = build_amount_matrix_with_poi(train, poi, month_codes)
    a_pred = predict_horizon_optimized(a_tr, best_params)
    
    # Apply December bump
    mult = compute_december_multipliers(
        a_tr, 
        clip_low=best_params['clip_low'], 
        clip_high=best_params['clip_high']
    )
    for t in a_pred.index[a_pred.index % 12 == 11]:
        a_pred.loc[t] = apply_december_bump_row(a_pred.loc[t], mult)
    
    sub = build_submission_df(a_pred, test, month_codes)
    sub.to_csv('ewgm_optuna_optimized.csv', index=False)
    print(f"\nOptimized submission saved: ewgm_optuna_optimized.csv")
    return sub


# =====================================================
# Comparison with Baseline
# =====================================================

def compare_with_baseline(optimized_sub_path, baseline_sub_path):
    """Compare optimized submission with baseline."""
    print("\n" + "="*60)
    print("COMPARISON WITH BASELINE")
    print("="*60)
    
    baseline = pd.read_csv(baseline_sub_path)
    optimized = pd.read_csv(optimized_sub_path)
    
    baseline_col = baseline.columns[1]
    optimized_col = optimized.columns[1]
    
    # Correlation
    corr = baseline[baseline_col].corr(optimized[optimized_col])
    print(f"\nCorrelation with baseline: {corr:.4f}")
    
    # Differences
    diff = (optimized[optimized_col] - baseline[baseline_col]).abs()
    pct_change = diff / (baseline[baseline_col] + 1)
    
    print(f"Mean absolute difference: {diff.mean():.2f}")
    print(f"Median absolute difference: {diff.median():.2f}")
    print(f"Mean % change: {pct_change.mean()*100:.1f}%")
    print(f"Predictions with >50% change: {(pct_change > 0.5).sum()} / {len(pct_change)}")
    
    if corr > 0.8:
        print("\n✓ HIGH CORRELATION: Predictions are similar - safe to submit")
    elif corr > 0.6:
        print("\n⚠ MODERATE CORRELATION: Predictions differ somewhat - medium risk")
    else:
        print("\n✗ LOW CORRELATION: Predictions differ significantly - high risk")


# =====================================================
# Main
# =====================================================

def main():
    print("Loading data...")
    month_codes = build_month_codes()
    train, test, poi = load_competition_data_with_poi()
    a_tr, poi_features = build_amount_matrix_with_poi(train, poi, month_codes)
    
    print(f"Data loaded: {len(a_tr)} months, {len(a_tr.columns)} sectors")
    
    # Run Optuna optimization
    best_params, best_score = run_optuna_optimization(a_tr, n_trials=100)
    
    # Save best parameters
    params_df = pd.DataFrame([best_params])
    params_df['estimated_score'] = best_score
    params_df.to_csv('optuna_best_params.csv', index=False)
    print("\nBest parameters saved to: optuna_best_params.csv")
    
    # Generate submission
    print("\n" + "="*60)
    print("GENERATING SUBMISSION")
    print("="*60)
    
    sub = generate_submission_optimized(best_params)
    
    print(f"\nPrediction stats:")
    print(f"  Total rows: {len(sub)}")
    print(f"  Mean: {sub['new_house_transaction_amount'].mean():.2f}")
    print(f"  Median: {sub['new_house_transaction_amount'].median():.2f}")
    print(f"  Min: {sub['new_house_transaction_amount'].min():.2f}")
    print(f"  Max: {sub['new_house_transaction_amount'].max():.2f}")
    
    # Compare with baseline (if available)
    # Uncomment and adjust path if you have baseline submission
    # compare_with_baseline(
    #     'ewgm_optuna_optimized.csv',
    #     '/path/to/baseline_submission.csv'
    # )


if __name__ == "__main__":
    main()

[I 2025-09-30 16:11:08,296] A new study created in memory with name: no-name-c07f4fb5-6471-4f85-8284-395e1b91ce46


Loading data...
Data loaded: 67 months, 96 sectors
OPTUNA OPTIMIZATION (100 trials)


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-09-30 16:11:08,357] Trial 0 finished with value: 41.84042690299551 and parameters: {'alpha': 0.5247240713084175, 'n_lags': 12, 'clip_low': 0.846398788362281, 'clip_high': 1.559195090518222, 't2': 4}. Best is trial 0 with value: 41.84042690299551.
[I 2025-09-30 16:11:08,376] Trial 1 finished with value: 42.579195370175064 and parameters: {'alpha': 0.3935967122017216, 'n_lags': 3, 'clip_low': 0.8732352291549871, 'clip_high': 1.5606690070459253, 't2': 10}. Best is trial 0 with value: 41.84042690299551.
[I 2025-09-30 16:11:08,398] Trial 2 finished with value: 42.775131639182995 and parameters: {'alpha': 0.31235069657748143, 'n_lags': 12, 'clip_low': 0.8664885281600844, 'clip_high': 1.3274034664069656, 't2': 4}. Best is trial 0 with value: 41.84042690299551.
[I 2025-09-30 16:11:08,415] Trial 3 finished with value: 42.372669480541894 and parameters: {'alpha': 0.41004270591206027, 'n_lags': 6, 'clip_low': 0.8049512863264475, 'clip_high': 1.4591670111852695, 't2': 5}. Best is trial 0 w

In [14]:
import pandas as pd

# Load both submissions
baseline_sub = pd.read_csv('/Users/nikola/Python/KaggleCompetition/output/EWGM_POI/EWGM_POI_submission_poi.csv')
optimized_sub = pd.read_csv('/Users/nikola/Python/KaggleCompetition/output/LASSO/submission.csv')

# Check what columns exist
print("Baseline columns:", baseline_sub.columns.tolist())
print("Optimized columns:", optimized_sub.columns.tolist())

# The column is probably 'new_house_transaction_amount' in both
baseline_col = baseline_sub.columns[1]  # Second column (after 'id')
optimized_col = optimized_sub.columns[1]

print(f"\nUsing columns: '{baseline_col}' vs '{optimized_col}'")

# Check correlation
corr = baseline_sub[baseline_col].corr(optimized_sub[optimized_col])
print(f"\nCorrelation: {corr:.4f}")

# Check how different they are
diff = (optimized_sub[optimized_col] - baseline_sub[baseline_col]).abs()
print(f"Mean absolute difference: {diff.mean():.2f}")
print(f"Median absolute difference: {diff.median():.2f}")

# Percentage changes
pct_change = diff / (baseline_sub[baseline_col] + 1)  # +1 to avoid divide by zero
print(f"\nMean % change: {pct_change.mean()*100:.1f}%")
print(f"Predictions with >50% change: {(pct_change > 0.5).sum()} / {len(pct_change)}")
print(f"Predictions with >100% change: {(pct_change > 1.0).sum()} / {len(pct_change)}")

Baseline columns: ['id', 'new_house_transaction_amount']
Optimized columns: ['id', 'amount']

Using columns: 'new_house_transaction_amount' vs 'amount'

Correlation: 0.0159
Mean absolute difference: 29664.12
Median absolute difference: 17194.19

Mean % change: 489685.8%
Predictions with >50% change: 926 / 1152
Predictions with >100% change: 417 / 1152
